In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import Image
#Image(filename='test.jpg') 


In [2]:
!pip uninstall transformers -y
!pip install transformers
!pip install evaluate
!pip install tensorflow-datasets -U
#!python3 -m pip install rouge_score
!pip install rouge_score

Found existing installation: transformers 4.27.3
Uninstalling transformers-4.27.3:
  Successfully uninstalled transformers-4.27.3
  Using cached transformers-4.27.3-py3-none-any.whl (6.8 MB)


In [3]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
print(tf.test.gpu_device_name())
import transformers
transformers.__version__

2.6.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
/device:GPU:0


'4.27.3'

## 요약(Summarization)

### text summarization

#### 도서  자료 요약 말뭉치 로드

In [104]:
from datasets import load_dataset
dataset = load_dataset("./AI_HUB/aihub_book_summarization.py", data_dir="./AI_HUB")
dataset

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['passage_id', 'passage', 'summary'],
        num_rows: 160002
    })
    validation: Dataset({
        features: ['passage_id', 'passage', 'summary'],
        num_rows: 19999
    })
})

In [105]:
dataset['train'][0]
dataset['train'][:]

{'passage_id': 'PCY_201410221112092321_0',
 'passage': '지금의 국가가 직면하는 위기는 전통사회의 그것과 위기의 규모뿐만아니라 종류와 성질 면에서 사뭇 다르다고 할 수 있다. 즉 현대 사회에서는 매우 다양하고 광범위하게 위기가 발생할 뿐만 아니라 지구온난화에 따른 각종 자연재해로부터 인위적 재난인 사회적 이슈로 인한 위기에 이르기까지 복잡한 양상을 보이고 있다. 따라서 위기의 영향은 한 산업뿐만 아니라 국가의 재난으로 이어지는 경우가 많다. 따라서 국가는 이러한 위기상황이 발생하였을 경우에 그 징후를 빨리 파악하고 대응하지 않으면 안 되는데, 국민의 생명과 재산을 보호하는 것이 바로 국가의 가장 기본적인 의무이기 때문이다.',
 'summary': '현대 사회에서 발생하는 다양하고 복잡한 위기는 과거와 다르게 국가의 재난으로 이어질 수 있으므로 국가의 신속한 대책 능력이 필요하다.'}

{'passage_id': ['PCY_201410221112092321_0',
  'PCY_201410221112092321_1',
  'PCY_201410221112092321_2',
  'PCY_201410221112092321_3',
  'PCY_201410221112092321_4',
  'PCY_201410221112520621_0',
  'PCY_201410221112520621_1',
  'PCY_201410221112520621_10',
  'PCY_201410221112520621_11',
  'PCY_201410221112520621_12',
  'PCY_201410221112520621_13',
  'PCY_201410221112520621_14',
  'PCY_201410221112520621_15',
  'PCY_201410221112520621_16',
  'PCY_201410221112520621_17',
  'PCY_201410221112520621_18',
  'PCY_201410221112520621_19',
  'PCY_201410221112520621_2',
  'PCY_201410221112520621_20',
  'PCY_201410221112520621_21',
  'PCY_201410221112520621_22',
  'PCY_201410221112520621_23',
  'PCY_201410221112520621_24',
  'PCY_201410221112520621_25',
  'PCY_201410221112520621_26',
  'PCY_201410221112520621_28',
  'PCY_201410221112520621_29',
  'PCY_201410221112520621_3',
  'PCY_201410221112520621_31',
  'PCY_201410221112520621_32',
  'PCY_201410221112520621_33',
  'PCY_201410221112520621_35',
  '

In [106]:
def show_samples(dataset, num_samples=3, seed=50):
    sample = dataset['train'].shuffle(seed=seed).select(range(num_samples))

    for example in sample:
        print(f"\n'>> passage: {example['passage']}'")
        print(f"\n'>> summary: {example['summary']}'")

show_samples(dataset)


'>> passage: 최근 들어 포스트차이나 신흥경제권으로 아세안의 4개국, 일명 ‘CLMV(캄보디아, 라오스, 미얀마, 베트남)’ 국가들이 부상하고 있다. 이들 국가 중 미얀마는 개방 정도가 미약하고 발전 수준이 낮아 아시아의 마지막 개척지로 일컬어지기도 하나 천연가스, 산림 등의 천연자원과 풍부한 저임노동력을 보유하고 있으며 중국, 인도 등 거대 소비시장과 인접하여 생산거점 및 물류 허브로서의 잠재력이 높은 것으로 평가된다. 특히 2011년 민간정부 출범 이후 국제사회의 대미얀마 경제제재 조치도 완화되어 EU와 미국으로의 수출 증대가 기대 되고 있으며 최근 중국, 베트남 등 주변국의 생산 비용이 가파르게 상승함에 따라 여러 선진국에서 신흥개발국가인 미얀마에 관심을 두고 있다. 이에 따라 우리나라도 풍부한 자원과 저임 노동력을 바탕으로 고성장이 기대되는 미얀마에 주목하고 우리기업의 글로벌 가치사슬 확대를 지원하고 동시에 이를 통해 미얀마의 산업 고도화에 기여할 수 있도록 양국 간 산업협력 기반 조성 방안을 모색해야 할 필요가 있다.'

'>> summary: 미얀마는 개방 정도가 적으나 천연자원이 풍부하고 임금이 낮아 잠재력이 높다. 또한 국제사회로부터의 경제제재도 완화되어 수출 증대및 고성장이 기대되는 만큼 양국간 협력 방안을 모색해야 한다.'

'>> passage: 한편 투입 쪽에서 보면 각 산업은 여러 가지 중간투입재화들과, 본원요소들, 간접세 등이 일정 비율로 결합되는 레온티에프 생산함수를 구성하고 있다. 그러나 투입요소별로 보면 중간투입재는 국내재 그리고 국내재와 대체가능한 수입재화로 구성되는 CES 함수이다. 요소수요 역시 여러 가지 본원적 요소들에 대한 결합수요로서 토지, 자본 그리고 노동의 CES 집계함수이다. 모든 산업들이 이처럼 공통적인 생산구조를 가지고 있지만 요소간 결합비율과 행태 파라미터들은 산업에 따라서 다를 수 있다. 이하에서는 중간재 투입에 대한 수요를 살펴보기로 한다.'

'>> summary: 투입 측면에서 살펴보면 각 

In [107]:
dataset.set_format("pandas")
df = dataset["train"][:]
df

,passage_id,passage,summary
0,PCY_201410221112092321_0,지금의 국가가 직면하는 위기는 전통사회의 그것과 위기의 규모뿐만아니라 종류와 성질 ...,현대 사회에서 발생하는 다양하고 복잡한 위기는 과거와 다르게 국가의 재난으로 이어질...
1,PCY_201410221112092321_1,자연재해나 불량 수산식품 파동 등의 인위적 재난이 수산분야 피해로이어질 경우 일반분...,"수산분야의 인위적 재난 시수산물 수급 문제로 생산자들의 매출이 크게 줄고, 시장 전..."
2,PCY_201410221112092321_2,하지만 이와 반대로 MG 파동 이후 기간(2005.10～2011.12)의 경우에는 ...,MG 파동 이후에는 양식 송어의 생산량의 증가가 통계적 유의성을 갖는 것으로 드러났...
3,PCY_201410221112092321_3,그러나 사회적 이슈가 발생하였을 경우에 전문적으로 이러한 정보를 수집하고 분석하는 ...,사회적 이슈 발생 시 전문적으로 정보 수집 및 분석을 하는 관측 시스템이 아직 구축...
4,PCY_201410221112092321_4,네 번째는 공급감소로 국내 물량이 부족하게 되면 유통질서가 왜곡될 가능성이 많기 때...,일부 경우에서의 유통질서 혼란을 방지하기 위해 거래형태의 변형이 필요하다. 의도적 ...
...,...,...,...
159997,PCY_202005110229522661_8,지도자로 등장한 2012년부터 2016년까지는 최고지도자로 추대되기 위한 준비기간이...,2012년 김정은은 국가 운영과 관련해 퍼포먼스를 선보였다. LED 패널을 이용한 ...
159998,PCY_202005120306026091_0,그러나 공공도서관은 과거 도서출판의 급속한 성장에 따른 장서수의 확충과 미디어의 확...,양적 성장을 추구했던 공공도서관은 이젠 질적 향상에도 집중해야 할 때이다. 정보통신...
159999,PCY_202005120306026091_1,"작은도서관의 이용자가 이동에 제약이 있는 유아, 노인들처럼 도보만을 이용할 수 있는...",작은도서관 이용자가 도보이동반경 내에 다수의 도서관이 없는 한 대체재의 역할을 수행...
160000,PCY_202005120306026091_2,"공공도서관 사서는 정보가 축적된 모든 형태의 매체자료(인쇄자료, 필사자료, 시청각자...","공공도서관 사서는 정보자료를 국민에게 제공하며 정보자료의 유통, 정보 접근 및 이용..."


#### 임시 : 테스트를 위해 sample 크기 줄임

In [51]:
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict

dataset.set_format("pandas")
df_t = dataset["train"][:10000]
df_v = dataset["validation"][:1000]
# 최상위 20개의 상품에 대한 개수 보여주기...
df_t = Dataset.from_pandas(df_t)
df_v = Dataset.from_pandas(df_v)

dataset_s = DatasetDict()


dataset_s['train'] = df_t
dataset_s['validation'] = df_v
dataset_s
dataset = dataset_s

DatasetDict({
    train: Dataset({
        features: ['passage_id', 'passage', 'summary'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['passage_id', 'passage', 'summary'],
        num_rows: 1000
    })
})

In [109]:
dataset.reset_format()
dataset

DatasetDict({
    train: Dataset({
        features: ['passage_id', 'passage', 'summary'],
        num_rows: 160002
    })
    validation: Dataset({
        features: ['passage_id', 'passage', 'summary'],
        num_rows: 19999
    })
})

In [110]:
dataset.keys()
dataset['train'][0].keys()
#dataset.filter()

dict_keys(['train', 'validation'])

dict_keys(['passage_id', 'passage', 'summary'])

### 텍스트 요약 모델 :  ke-t5

### Auto로 사용하기 (우선 이거 사용해보자)

In [122]:
from transformers import AutoModel, AutoTokenizer

#model_checkpoint = "KETI-AIR/ke-t5-small-ko"
model_checkpoint = "data/test"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModel.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at data/test were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#### 좀 더 알아보기

In [17]:
from transformers import T5TokenizerFast, T5ForConditionalGeneration

#model_checkpoint = "KETI-AIR/ke-t5-small-ko"
model_checkpoint = "data/test"

tokenizer = T5TokenizerFast.from_pretrained(model_checkpoint)
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)

In [112]:
model

T5Model(
  (shared): Embedding(64128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(64128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): Linear(in_features=

### 데이터 전처리하기

In [123]:
from transformers import AutoTokenizer

#model_checkpoint = "KETI-AIR/ke-t5-small-ko"
model_checkpoint = "data/test"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [117]:
inputs = tokenizer("그는 더 이상 말을 하지 않았지만 우리는 항상 유난히 절제된 방식으로 의사소통을 해왔고 나는 그가\
그 이상을 의미한다는 것을 이해했습니다. 결과적으로 나는 모든 판단을 유보하는 경향이 있으\
며, 그 습관은 나에게 많은 호기심을 불러일으켰고 또한 나를 적지 않은 노련한 지루한 사람의\
희생자로 만들었습니다. 비정상적인 마음은 정상적인 사람에게 나타날 때 이 특성을 빠르게 감지하고\
부착합니다. 그래서 대학에서 저는 부당하게 정치인이라는 비난을 받았습니다. 남자들. 대부분\
의 비밀은 구하지 않은 것입니다. 은밀한 계시가 수평선에서 떨고 있다는 확실한 신호를 깨달\
았을 때 나는 자주 잠, 몰두 또는 적대적인 경솔함을 가장했습니다. 젊은 남성의 내밀한 폭로, 또는\
적어도 그들이 표현하는 용어는 대개 표절적이며 명백한 억압으로 손상되었습니다. 판단을\
유보하는 것은 무한한 희망의 문제입니다. 아버지가 속물스럽게 말했고 내가 속물스럽게 되풀이해서 말\
하듯이 기본적인 품위 감각은 태어날 때부터 불평등하게 나누어진다는 사실을 잊어버리면 여\
전히 무언가를 놓치는 것이 조금 두렵습니다.")
len(inputs)
inputs

2

{'input_ids': [188, 121, 276, 907, 751, 6276, 1380, 2678, 25746, 56816, 2313, 24428, 11, 7, 40613, 600, 1435, 775, 4867, 7859, 12, 233, 2292, 507, 3, 14464, 600, 350, 9492, 21123, 74, 11460, 773, 4, 70, 15277, 19, 8342, 253, 25266, 2695, 30, 7210, 61605, 25, 360, 2754, 4862, 450, 57152, 20416, 50, 1941, 811, 1027, 3226, 51758, 3, 51348, 16051, 13877, 6737, 11699, 172, 47, 9692, 3037, 9734, 72, 291, 4369, 1008, 3, 743, 20426, 1928, 30216, 7620, 308, 17718, 14757, 3, 24003, 3, 4744, 4848, 19, 48693, 450, 1738, 3, 47622, 14175, 22, 35, 1339, 29175, 45304, 408, 9964, 22781, 54094, 11, 172, 600, 2290, 4876, 4, 29943, 739, 47224, 52, 58879, 2273, 189, 507, 3, 2332, 19193, 21660, 50, 9197, 4, 739, 169, 13463, 5740, 17960, 11418, 12, 13014, 25966, 23441, 19239, 18113, 45, 11067, 31601, 3, 9492, 514, 518, 74, 167, 30195, 26855, 776, 335, 3, 9222, 815, 801, 7866, 21200, 712, 815, 801, 7866, 14891, 869, 372, 29039, 11556, 41025, 11412, 19, 48802, 5461, 18659, 180, 12566, 3998, 12, 1205, 49863, 16

In [118]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['▁그는',
 '▁더',
 '▁이상',
 '▁말을',
 '▁하지',
 '▁않았지만',
 '▁우리는',
 '▁항상',
 '▁유난히',
 '▁절제된',
 '▁방식으로',
 '▁의사소통',
 '을',
 '▁',
 '해왔고',
 '▁나는',
 '▁그가',
 '그',
 '▁이상을',
 '▁의미한다',
 '는',
 '▁것을',
 '▁이해',
 '했습니다',
 '.',
 '▁결과적으로',
 '▁나는',
 '▁모든',
 '▁판단을',
 '▁유보',
 '하는',
 '▁경향이',
 '▁있으며',
 ',',
 '▁그',
 '▁습관',
 '은',
 '▁나에게',
 '▁많은',
 '▁호기심을',
 '▁불러',
 '일',
 '으',
 '켰',
 '고',
 '▁또한',
 '▁나를',
 '▁적지',
 '▁않은',
 '▁노련한',
 '▁지루',
 '한',
 '▁사람의',
 '희',
 '생',
 '자로',
 '▁만들었습니다',
 '.',
 '▁비정상적인',
 '▁마음은',
 '▁정상적인',
 '▁사람에게',
 '▁나타날',
 '▁때',
 '▁이',
 '▁특성을',
 '▁빠르게',
 '▁감지',
 '하고',
 '부',
 '착',
 '합니다',
 '.',
 '▁그래서',
 '▁대학에서',
 '▁저는',
 '▁부당하게',
 '▁정치인',
 '이라는',
 '▁비난을',
 '▁받았습니다',
 '.',
 '▁남자들',
 '.',
 '▁대부분의',
 '▁비밀',
 '은',
 '▁구하지',
 '▁않은',
 '▁것입니다',
 '.',
 '▁은밀한',
 '▁계시',
 '가',
 '▁수',
 '평',
 '선에서',
 '▁떨고',
 '▁있다는',
 '▁확실한',
 '▁신호를',
 '▁깨달았',
 '을',
 '▁때',
 '▁나는',
 '▁자주',
 '▁잠',
 ',',
 '▁몰두',
 '▁또는',
 '▁적대적',
 '인',
 '▁경솔',
 '함을',
 '▁가장',
 '했습니다',
 '.',
 '▁젊은',
 '▁남성의',
 '▁내밀',
 '한',
 '▁폭로',
 ',',
 '▁또는',
 '적',
 '어도',
 '▁

#### Tokenization 

In [119]:
#max_input_length = 512
#max_target_length = 30
#max_input_length = 512*2
#max_target_length = 512
max_input_length = 512
max_target_length = 60

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["passage"], max_length=max_input_length, truncation=True
    )
    # 타겟을 위한 토크나이저 설정
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [120]:
dataset
preprocess_function(dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['passage_id', 'passage', 'summary'],
        num_rows: 160002
    })
    validation: Dataset({
        features: ['passage_id', 'passage', 'summary'],
        num_rows: 19999
    })
})

/opt/conda/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [9046, 10755, 34760, 12, 3629, 12, 3209, 23857, 9398, 31, 38538, 2730, 2486, 7793, 222, 8753, 41, 19255, 8060, 31465, 5007, 25, 145, 35, 32, 3, 1553, 1745, 12846, 12, 1034, 28093, 39450, 21095, 5767, 2474, 309, 59119, 9, 725, 1053, 44818, 1074, 46780, 5930, 52, 1574, 33094, 1836, 6826, 11430, 10159, 22710, 1340, 32, 3, 1433, 38538, 20589, 59, 1282, 2486, 309, 7128, 5930, 45, 5817, 2307, 1542, 3, 1433, 26035, 902, 52379, 15, 1400, 48162, 8698, 70, 26195, 21, 2510, 10513, 1978, 218, 2418, 150, 7593, 4, 2762, 28979, 13945, 27592, 179, 584, 7128, 189, 11556, 4939, 2992, 501, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [1745, 12846, 5127, 28093, 10159, 3629, 12, 20068, 8998, 7128,

In [121]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/160002 [00:00<?, ? examples/s]

Map:   0%|          | 0/19999 [00:00<?, ? examples/s]

### 텍스트 요약을 위한 평가지표(metrics)

In [124]:
generated_summary = "나는 정말로 진짜로 헝거 게임 읽는거 좋아해"
reference_summary = "나는 헝거 게임 읽는거 사랑해"

In [125]:
generated_summary_ids = tokenizer("나는 정말로 진짜로 헝거 게임 읽는거 좋아해")
generated_summary_ids['input_ids']

reference_summary_ids = tokenizer("나는 헝거 게임 읽는거 사랑해")
reference_summary_ids['input_ids']

tokenizer.convert_ids_to_tokens(generated_summary_ids.input_ids)
tokenizer.convert_ids_to_tokens(reference_summary_ids.input_ids)

[600, 17747, 2160, 34, 7, 46145, 923, 1352, 12349, 923, 36677, 1]

[600, 7, 46145, 923, 1352, 12349, 923, 20304, 1]

['▁나는', '▁정말로', '▁진짜', '로', '▁', '헝', '거', '▁게임', '▁읽는', '거', '▁좋아해', '</s>']

['▁나는', '▁', '헝', '거', '▁게임', '▁읽는', '거', '▁사랑해', '</s>']

In [126]:
from datasets import load_metric

rouge_score = load_metric("rouge")

In [127]:
scores = rouge_score.compute(
    predictions=[generated_summary_ids], references=[reference_summary_ids]
)
scores
scores['rouge1']
scores['rouge2']
scores['rougeL']
scores['rougeLsum']

{'rouge1': AggregateScore(low=Score(precision=0.75, recall=0.9545454545454546, fmeasure=0.84), mid=Score(precision=0.75, recall=0.9545454545454546, fmeasure=0.84), high=Score(precision=0.75, recall=0.9545454545454546, fmeasure=0.84)),
 'rouge2': AggregateScore(low=Score(precision=0.6666666666666666, recall=0.8571428571428571, fmeasure=0.75), mid=Score(precision=0.6666666666666666, recall=0.8571428571428571, fmeasure=0.75), high=Score(precision=0.6666666666666666, recall=0.8571428571428571, fmeasure=0.75)),
 'rougeL': AggregateScore(low=Score(precision=0.75, recall=0.9545454545454546, fmeasure=0.84), mid=Score(precision=0.75, recall=0.9545454545454546, fmeasure=0.84), high=Score(precision=0.75, recall=0.9545454545454546, fmeasure=0.84)),
 'rougeLsum': AggregateScore(low=Score(precision=0.75, recall=0.9545454545454546, fmeasure=0.84), mid=Score(precision=0.75, recall=0.9545454545454546, fmeasure=0.84), high=Score(precision=0.75, recall=0.9545454545454546, fmeasure=0.84))}

AggregateScore(low=Score(precision=0.75, recall=0.9545454545454546, fmeasure=0.84), mid=Score(precision=0.75, recall=0.9545454545454546, fmeasure=0.84), high=Score(precision=0.75, recall=0.9545454545454546, fmeasure=0.84))

AggregateScore(low=Score(precision=0.6666666666666666, recall=0.8571428571428571, fmeasure=0.75), mid=Score(precision=0.6666666666666666, recall=0.8571428571428571, fmeasure=0.75), high=Score(precision=0.6666666666666666, recall=0.8571428571428571, fmeasure=0.75))

AggregateScore(low=Score(precision=0.75, recall=0.9545454545454546, fmeasure=0.84), mid=Score(precision=0.75, recall=0.9545454545454546, fmeasure=0.84), high=Score(precision=0.75, recall=0.9545454545454546, fmeasure=0.84))

AggregateScore(low=Score(precision=0.75, recall=0.9545454545454546, fmeasure=0.84), mid=Score(precision=0.75, recall=0.9545454545454546, fmeasure=0.84), high=Score(precision=0.75, recall=0.9545454545454546, fmeasure=0.84))

In [128]:
scores['rouge2'].low
scores["rouge2"].mid
scores['rouge2'].high

Score(precision=0.6666666666666666, recall=0.8571428571428571, fmeasure=0.75)

Score(precision=0.6666666666666666, recall=0.8571428571428571, fmeasure=0.75)

Score(precision=0.6666666666666666, recall=0.8571428571428571, fmeasure=0.75)

### 강력한 베이스라인 모델 만들기

In [129]:
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to /aiffel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [130]:
from nltk.tokenize import sent_tokenize


def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])


print(three_sentence_summary(dataset["train"][1]["passage"]))
print()
print(dataset["train"][1]["summary"])

자연재해나 불량 수산식품 파동 등의 인위적 재난이 수산분야 피해로이어질 경우 일반분야와 마찬가지로 생산량 감소 혹은 수요 감소와 같은 수산물 수급에 문제가 발생하게 된다.
그리고 이러한 수급문제가 발생할 경우 생산량이 감소하거나 수요 감소에 따라 생산자들의 매출이 크게 줄어들 수 있으며, 시장 전체에 있어서의 생산자 잉여는 감소하게 된다.
마찬가지로 소비자의 경우에도 생산량이 감소할 경우 보다 높은 시장가격을 지불해야함에 따라 소비자 잉여는 감소하게 된다.

수산분야의 인위적 재난 시수산물 수급 문제로 생산자들의 매출이 크게 줄고, 시장 전체의 생산자 잉여는 감소하며 그에 따른 높은 시장가격을 지불해야하므로 소비자 잉여도 감소하게 된다. 


In [131]:
def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(text) for text in dataset["passage"]]
    return metric.compute(predictions=summaries, references=dataset["summary"])

In [134]:
import pandas as pd

score = evaluate_baseline(dataset["validation"], rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(score[rn].mid.fmeasure * 100, 2)) for rn in rouge_names)
rouge_dict

{'rouge1': 15.61, 'rouge2': 5.43, 'rougeL': 15.36, 'rougeLsum': 15.15}

### rouge (구현중)

In [133]:
dataset['validation']
dataset['validation'][0]['passage']
dataset['validation'][0]['summary']
#metric.compute(predictions=dataset['validation'][0]['passage'], references=dataset['validation'][0]['summary'])


Dataset({
    features: ['passage_id', 'passage', 'summary'],
    num_rows: 19999
})

"7) 실시계약의 기간\n일곱째 요소는 '특허권의 존속기간과 실시권의 기간'이다. 대개는 계약기간이 길수록 실시료가 높을 것이지만 항상 그런 것은 아니다. 어떤 기술들은 상대적으로 짧은, 특허기간보다 훨씬 짧은 존속기간을 가지기 때문이며, 따라서 생각보다 이 요소가 중요하지 않을 수 있다. \n8) 특허제품의 수익성과 인기\n여덟째 요소는 '특허발명의 실시로 제조된 제품의 확립된 수익성 및 그 사업상의 성공도, 현재의 인기도'이다.\n미국의 판례는 상업적 성공이라는 논점을 다루면서 문제는 특허청구된 대상이라는 점을 명확하게 하여 왔다. 그러나 침해품에 따라서는 확립된 수익성이 존재하지 않는 경우도 있을 수 있고, 상품의 시장개척 초기 과정에서는 수익성을 희생하는 경우도 있을 수 있다. \n9) 특허제품의 유용성과 유리함\n아홉째 요소는 '유사한 결과를 실현하기 위하여 사용되는 구 방식이나 기기(혹시 있다면)에 대한 특허제품의 실용성 및 이점'이다. 합리적 실시료는 특허발명이 실시될 때 얻어지는 이득에 기초하여야 한다는 관념을 반영한다. 만약 협상이 깨진다면 실시료 상당액은 협상의 양 당사자가 실시료 약정에 응할 이유를 가져야 하는 그러한 경제적 가치의 할당된 비율을 나타내야만 한다고 한다. \n10) 특허발명의 성질과 특징\n열째 요소는 '특허발명의 성질', 실시허락권자에 의하여 소유 또는 생산된 특허발명의 상업적으로 구체화된 성격 및 그 발명을 사용해 온 자가 받은 편익'이다. 이 요소는 흔히 특허발명의 전반적인 상업화와 관련하여 논하여진다. "

'특허제품의 유용성과 유리함은 유사한 결과를 실현하기 위해 사용되는 구 방식이나 기기에 대한 특허제품의 실용성 및 이점으로 합리적 실시료는 특허발명이 실시될 때 얻어지는 이득에 기초하여야 한다는 관념을 반영하고 만일 협상이 깨진다면 실시료 상당액은 실시료 약정에 응할 이유를 가져야 하는 경제적 가치의 할당된 비율을 나타내야만 한다는 것이다. '

### Trainer API를 이용하여 mT5 미세조정

In [135]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

model_checkpoint
tokenized_datasets['train'][0]


tokenized_datasets['train']
len(tokenized_datasets['train']) // 32 #batch_size

'data/test'

{'passage_id': 'PCY_201410221112092321_0',
 'passage': '지금의 국가가 직면하는 위기는 전통사회의 그것과 위기의 규모뿐만아니라 종류와 성질 면에서 사뭇 다르다고 할 수 있다. 즉 현대 사회에서는 매우 다양하고 광범위하게 위기가 발생할 뿐만 아니라 지구온난화에 따른 각종 자연재해로부터 인위적 재난인 사회적 이슈로 인한 위기에 이르기까지 복잡한 양상을 보이고 있다. 따라서 위기의 영향은 한 산업뿐만 아니라 국가의 재난으로 이어지는 경우가 많다. 따라서 국가는 이러한 위기상황이 발생하였을 경우에 그 징후를 빨리 파악하고 대응하지 않으면 안 되는데, 국민의 생명과 재산을 보호하는 것이 바로 국가의 가장 기본적인 의무이기 때문이다.',
 'summary': '현대 사회에서 발생하는 다양하고 복잡한 위기는 과거와 다르게 국가의 재난으로 이어질 수 있으므로 국가의 신속한 대책 능력이 필요하다.',
 'input_ids': [9046,
  10755,
  34760,
  12,
  3629,
  12,
  3209,
  23857,
  9398,
  31,
  38538,
  2730,
  2486,
  7793,
  222,
  8753,
  41,
  19255,
  8060,
  31465,
  5007,
  25,
  145,
  35,
  32,
  3,
  1553,
  1745,
  12846,
  12,
  1034,
  28093,
  39450,
  21095,
  5767,
  2474,
  309,
  59119,
  9,
  725,
  1053,
  44818,
  1074,
  46780,
  5930,
  52,
  1574,
  33094,
  1836,
  6826,
  11430,
  10159,
  22710,
  1340,
  32,
  3,
  1433,
  38538,
  20589,
  59,
  1282,
  2486,
  309,
  7128,
  5930,
  45,
  5817,
 

Dataset({
    features: ['passage_id', 'passage', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 160002
})

5000

In [137]:
model_checkpoint.split("/")[-1]

'test'

#### 전체 sample사용 시

In [140]:
from transformers import Seq2SeqTrainingArguments

batch_size = 16
num_train_epochs = 3
# 매 에포크마다 training loss를 보여준다.
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir='./data',
    evaluation_strategy="steps",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
#    logging_steps=logging_steps,
#    push_to_hub=True,
    save_strategy = 'steps',
    group_by_length = True,
)

In [139]:
### #### 임시 : 테스트를 위해 sample 크기 줄임 사용 시

In [74]:
from transformers import Seq2SeqTrainingArguments

batch_size = 16
num_train_epochs = 3
# 매 에포크마다 training loss를 보여준다.
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir='./data',
    evaluation_strategy='epoch',
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
#    logging_steps=logging_steps,
#    push_to_hub=True,
    save_strategy = 'epoch',
    group_by_length = True,
)

In [141]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # 생성된 요약을 텍스트로 디코딩
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # 레이블 내의 -100을 교체한다.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # 레퍼런스 요약을 텍스트로 디코딩
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE는 각 문장 다음에 개행문자를 요구한다.
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    #print("Predict", decoded_preds)
    #print("Label",decoded_labels)
    # ROUGE 점수 계산
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # 중간 점수(median scores) 추출
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [142]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [143]:
dataset["train"].column_names

tokenized_datasets = tokenized_datasets.remove_columns(
    dataset["train"].column_names
)

['passage_id', 'passage', 'summary']

In [144]:
features = [tokenized_datasets["train"][i] for i in range(2)]
#features
data_collator(features)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[ 9046, 10755, 34760,    12,  3629,    12,  3209, 23857,  9398,    31,
         38538,  2730,  2486,  7793,   222,  8753,    41, 19255,  8060, 31465,
          5007,    25,   145,    35,    32,     3,  1553,  1745, 12846,    12,
          1034, 28093, 39450, 21095,  5767,  2474,   309, 59119,     9,   725,
          1053, 44818,  1074, 46780,  5930,    52,  1574, 33094,  1836,  6826,
         11430, 10159, 22710,  1340,    32,     3,  1433, 38538, 20589,    59,
          1282,  2486,   309,  7128,  5930,    45,  5817,  2307,  1542,     3,
          1433, 26035,   902, 52379,    15,  1400, 48162,  8698,    70, 26195,
            21,  2510, 10513,  1978,   218,  2418,   150,  7593,     4,  2762,
         28979, 13945, 27592,   179,   584,  7128,   189, 11556,  4939,  2992,
           501,     3,     1,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [44818,   116, 16132, 23344,  5348, 29405,   364, 

In [145]:
tokenized_datasets["train"]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 160002
})

In [146]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [148]:
trainer.train()

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
500,2.879300,2.886991,9.707700,2.093600,9.629400,9.638400
1000,2.809100,2.860319,9.771500,2.004000,9.717300,9.713700
1500,2.801300,2.844328,9.875900,2.106400,9.819900,9.810000
2000,2.775900,2.840902,9.977900,2.159200,9.919700,9.926600
2500,2.774700,2.813546,9.808700,2.033300,9.749200,9.737000
3000,2.754300,2.820667,10.180400,2.187200,10.104100,10.110500
3500,2.727500,2.813383,9.768200,2.136900,9.711600,9.690500
4000,2.737400,2.785702,10.036300,2.160200,9.976100,9.961300
4500,2.720400,2.769189,10.267100,2.259700,10.207800,10.195200
5000,2.708200,2.754717,10.143800,2.195800,10.072000,10.063800


TrainOutput(global_step=30003, training_loss=2.5805145565702117, metrics={'train_runtime': 46560.7211, 'train_samples_per_second': 10.309, 'train_steps_per_second': 0.644, 'total_flos': 2.2434538791297024e+16, 'train_loss': 2.5805145565702117, 'epoch': 3.0})

In [149]:
trainer.evaluate()

{'eval_loss': 2.56227445602417,
 'eval_rouge1': 11.0005,
 'eval_rouge2': 2.46,
 'eval_rougeL': 10.9431,
 'eval_rougeLsum': 10.935,
 'eval_runtime': 620.7399,
 'eval_samples_per_second': 32.218,
 'eval_steps_per_second': 2.014,
 'epoch': 3.0}

### 미세 조정된 모델 사용하기

In [153]:
model.save_pretrained('data/test_2')
tokenizer.save_pretrained('data/test_2')

('data/test_2/tokenizer_config.json',
 'data/test_2/special_tokens_map.json',
 'data/test_2/spiece.model',
 'data/test_2/added_tokens.json',
 'data/test_2/tokenizer.json')

### 좀 더 알아보기 

In [154]:
from transformers import T5TokenizerFast, T5ForConditionalGeneration

#model_checkpoint = "KETI-AIR/ke-t5-small-ko"
model_checkpoint = "data/test_2"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)

In [ ]:
#from transformers import AutoModelForSeq2SeqLM
#model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

### auto 사용

In [77]:
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained('data/test_2')
tokenizer = AutoTokenizer.from_pretrained('data/test_2')

Some weights of the model checkpoint at data/test were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### pipeline으로 확인

In [199]:
from transformers import pipeline

#hub_model_id = 'KETI-AIR/ke-t5-small-ko'
hub_model_id = 'data/test_2'
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

In [188]:
min_length=10
max_length=1024

summarizer('우리 가족은 3대에 걸쳐 이 중서부 도시에서 유명하고 유복한 사람들이었습니다.\
Carraways는 일종의 클랜이며 Dukes의 후손이라는 전통이 있습니다.\
Buccleuch, 하지만 내 라인의 실제 설립자는 51세에 이곳에 와서 남북 전쟁에 대리자를\
보냈고 오늘날 아버지가 수행하는 도매 하드웨어 사업을 시작한 할아버지의 형제였습\
니다.\
나는 이 종조부를 한 번도 본 적이 없지만 아버지 사무실에 걸려 있는 하드보일드 그림을 특\
히 언급하면서 나는 그와 닮았다고 생각합니다. 나는 1915년에 뉴헤이븐을 졸업했는\
데, 이는 아버지보다 25년이 지난 후였고, 조금 후에 나는 대전으로 알려진 게르만 민족의 지연\
된 이주에 참여했습니다. 반격이 너무 좋아서 정신없이 돌아왔습니다. 중서부는 세상\
의 따뜻한 중심이 아니라 이제 우주의 너덜너덜한 가장자리처럼 느껴졌습니다. 그래서 저\
는 동부로 가서 채권 사업을 배우기로 결정했습니다. 내가 아는 모든 사람들이 채권 사업에 종\
사하고 있었기 때문에 그것이 한 명의 독신자를 더 지원할 수 있을 것이라고 생각했습니다.\
이모와 삼촌은 모두 나를 위해 예비 학교를 선택하는 것처럼 그것에 대해 이야기하고 마침내 매우\
무겁고 주저하는 얼굴로 "왜‑예‑에스"라고 말했습니다. 아버지는 나에게 1년 동안 자금을 대는\
데 동의했고 여러 가지 지연 끝에 나는 22살 봄에 영구적으로 동부로 왔다고 생각했다.\
현실적인 것은 도시에서 방을 구하는 것이었지만, 따뜻한 계절이었고, 넓은 잔디밭과 친근\
한 나무가 있는 나라를 막 떠났기 때문에 사무실에 있는 청년이 출퇴근길에 함께 집\
을 구하자고 제안했을 때 좋은 생각인 것 같았습니다. 그는 한 달에 80에 비바람에 시달린\
골판지 방갈로 집을 찾았지만 막판에 회사는 그를 워싱턴으로 명령했고 나는 혼자 시골로 나\
갔습니다. 나는 개가 있었다. 적어도 그가 도망칠 때까지 며칠 동안은 개를 키웠고, 나이든\
Dodge와 내 침대를 정리하고 아침을 요리하고 전기 스토브 위에서 혼잣말로 핀란드의 지혜를\
중얼거리는 핀란드 여자가 있었다.\
하루 정도 외로웠는데 어느 날 아침 나보다 더 늦게 도착한 어떤 남자가 길에서 나를\
세웠다.\
"웨스트 에그 마을에 어떻게 가나요?" 그는 힘 없이 물었다.\
나는 그에게 말했다. 그리고 걸어가면서 더 이상 외롭지 않았습니다. 나는 안내자, 길잡이,\
최초의 정착민이었습니다. 그는 우연히 나에게 이웃의 자유를 부여했습니다.\
그래서 빠른 영화에서 사물이 자라는 것처럼 햇빛과 나무에서 자라는 거대한 잎사귀와 함께 나\
는 인생이 여름과 함께 다시 시작된다는 친숙한 확신을 가졌습니다.\
우선 읽을거리가 너무 많았고, 숨을 쉬게 하는 젊은 공기에서 뽑아내야 할 훌륭한 건강이 너무\
많았습니다. 나는 은행, 신용, 투자 증권에 관한 책을 12권 샀고, 그 책들은 마이다스, 모\
건, 메세나스만이 알고 있는 빛나는 비밀을 밝히겠다고 약속하며 조폐국에서 나온 새 돈처\
럼 붉은색과 금색으로 제 책장에 서 있었습니다. 그리고 그 외에도 많은 책을 읽어보고\
싶다는 높은 의욕을 가지고 있었습니다. 나는 대학에서 오히려 문학적이었다. 어느 해 나는 Yale\
News에 매우 엄숙하고 분명한 일련의 사설을 썼다. 다재다능한 남자.”\
이것은 단순한 에피그램이 아닙니다. 인생은 훨씬 더 성공적으로 바라볼 수 있습니다.\
예의 베이 건너편에는 패셔너블한 이스트 에그의 하얀 궁전이 물가를 따라 반짝이고 있었\
고, 여름의 역사는 내가 Tom Buchanans와 저녁 식사를 하기 위해 차를 몰고 그곳으로\
간 저녁부터 시작되었습니다. Daisy는 한 번 제거된 내 두 번째 사촌이었고 Tom\
은 대학에서 알고 있었습니다. 그리고 전쟁 직후 나는 시카고에서 그들과 함께 이틀을 보냈습니다.\
그녀의 남편은 다양한 신체적 업적 중에서 뉴 헤이븐에서 축구를 한 가장 강력한 목표 중 하나였습니\
다. 어떤 의미에서 국가적 인물, 21세에 극도로 제한적인 우수성에 도달한 남자 중 한 명입\
니다. 안티 클라이맥스. 그의 가족은 엄청나게 부유했지만(대학 시절에도 돈이 있는 그의 자\
유는 비난받아 마땅한 문제였습니다) 이제 그는 시카고를 떠나 숨이 멎을 듯한 방식으로 동부\
로 왔습니다. Lake Forest의 폴로 조랑말. 우리 세대의 남자가 그렇게 할 만큼 부유하다는\
것을 깨닫는 것은 어려웠습니다.', min_length=min_length, max_length=max_length)

Your max_length is set to 1024, but you input_length is only 793. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=396)


[{'summary_text': '우리 가족은 3대 중 중 중에서 유명하고 유복한 사람들이었습니다. Carraways는 클랜이며 Dukes의 후손이라 하였고, 내 라인의 설립자는 51세, 그의 설립자는 51세, 그의 설립자는 51세, 그의 설립자는 할아버지의 형제이다.'}]

In [187]:
min_length=10
max_length=1024

summarizer('우리 가족은 3대에 걸쳐 이 중서부 도시에서 유명하고 유복한 사람들이었습니다.\
Carraways는 일종의 클랜이며 Dukes의 후손이라는 전통이 있습니다.\
Buccleuch, 하지만 내 라인의 실제 설립자는 51세에 이곳에 와서 남북 전쟁에 대리자를\
보냈고 오늘날 아버지가 수행하는 도매 하드웨어 사업을 시작한 할아버지의 형제였습\
니다.\
나는 이 종조부를 한 번도 본 적이 없지만 아버지 사무실에 걸려 있는 하드보일드 그림을 특\
히 언급하면서 나는 그와 닮았다고 생각합니다. 나는 1915년에 뉴헤이븐을 졸업했는\
데, 이는 아버지보다 25년이 지난 후였고, 조금 후에 나는 대전으로 알려진 게르만 민족의 지연\
된 이주에 참여했습니다. 반격이 너무 좋아서 정신없이 돌아왔습니다. 중서부는 세상\
의 따뜻한 중심이 아니라 이제 우주의 너덜너덜한 가장자리처럼 느껴졌습니다. 그래서 저\
는 동부로 가서 채권 사업을 배우기로 결정했습니다. 내가 아는 모든 사람들이 채권 사업에 종\
사하고 있었기 때문에 그것이 한 명의 독신자를 더 지원할 수 있을 것이라고 생각했습니다.\
이모와 삼촌은 모두 나를 위해 예비 학교를 선택하는 것처럼 그것에 대해 이야기하고 마침내 매우\
무겁고 주저하는 얼굴로 "왜‑예‑에스"라고 말했습니다. 아버지는 나에게 1년 동안 자금을 대는\
데 동의했고 여러 가지 지연 끝에 나는 22살 봄에 영구적으로 동부로 왔다고 생각했다.\
현실적인 것은 도시에서 방을 구하는 것이었지만, 따뜻한 계절이었고, 넓은 잔디밭과 친근\
한 나무가 있는 나라를 막 떠났기 때문에 사무실에 있는 청년이 출퇴근길에 함께 집\
을 구하자고 제안했을 때 좋은 생각인 것 같았습니다. 그는 한 달에 80에 비바람에 시달린\
골판지 방갈로 집을 찾았지만 막판에 회사는 그를 워싱턴으로 명령했고 나는 혼자 시골로 나\
갔습니다. 나는 개가 있었다. 적어도 그가 도망칠 때까지 며칠 동안은 개를 키웠고, 나이든\
Dodge와 내 침대를 정리하고 아침을 요리하고 전기 스토브 위에서 혼잣말로 핀란드의 지혜를\
중얼거리는 핀란드 여자가 있었다.\
하루 정도 외로웠는데 어느 날 아침 나보다 더 늦게 도착한 어떤 남자가 길에서 나를\
세웠다.\
"웨스트 에그 마을에 어떻게 가나요?" 그는 힘 없이 물었다.\
나는 그에게 말했다. 그리고 걸어가면서 더 이상 외롭지 않았습니다. 나는 안내자, 길잡이,\
최초의 정착민이었습니다. 그는 우연히 나에게 이웃의 자유를 부여했습니다.\
그래서 빠른 영화에서 사물이 자라는 것처럼 햇빛과 나무에서 자라는 거대한 잎사귀와 함께 나\
는 인생이 여름과 함께 다시 시작된다는 친숙한 확신을 가졌습니다.\
우선 읽을거리가 너무 많았고, 숨을 쉬게 하는 젊은 공기에서 뽑아내야 할 훌륭한 건강이 너무\
많았습니다. 나는 은행, 신용, 투자 증권에 관한 책을 12권 샀고, 그 책들은 마이다스, 모\
건, 메세나스만이 알고 있는 빛나는 비밀을 밝히겠다고 약속하며 조폐국에서 나온 새 돈처\
럼 붉은색과 금색으로 제 책장에 서 있었습니다. 그리고 그 외에도 많은 책을 읽어보고\
싶다는 높은 의욕을 가지고 있었습니다. 나는 대학에서 오히려 문학적이었다. 어느 해 나는 Yale\
News에 매우 엄숙하고 분명한 일련의 사설을 썼다. 다재다능한 남자.”\
이것은 단순한 에피그램이 아닙니다. 인생은 훨씬 더 성공적으로 바라볼 수 있습니다.\
결국 단일 창에서.\
북미에서 가장 이상한 마을 중 한 곳에 집을 빌릴 수 있었던 것은 우연의 문제였습니다. 그곳\
은 뉴욕의 정동쪽으로 뻗어 있는 가냘프고 떠들썩한 섬에 있었는데, 그곳에는 다른 자연의\
진기함 중에서도 두 개의 특이한 지형이 있습니다. 도시에서 20마일 떨어진 곳에 거대한 알 한\
쌍이 서반구에서 가장 길들여진 염수인 Long Island Sound의 거대한 축축한 앞마당으로\
튀어나와 있습니다. 그것들은 콜럼버스 이야기에 나오는 달걀처럼 완벽한 타원형은 아니며,\
둘 다 접촉면이 납작하게 뭉개져 있습니다. 날개가 없는 개체에게 더 흥미로운 현상은 모양\
과 크기를 제외한 모든 세부 사항에서 유사하지 않다는 것입니다.\
나는 웨스트 에그에 살았는데, 그 둘 중 덜 유행했지만, 이것은 그들 사이의 기괴하고 적지 않\
은 불길한 대조를 표현하기 위한 가장 피상적인 꼬리표지만. 우리 집은 해협에서 불과 50\
야드 떨어진 알의 가장 끝에 있었고, 한 시즌에 12~15,000달러에 임대하는 두 개의 거대\
한 집 사이에 끼어 있었습니다. 내 오른쪽에 있는 것은 어떤 기준으로 보아도 어마어마한 것이었\
다. 노르망디에 있는 빌 드 빌 호텔을 사실적으로 모방한 것인데, 한쪽에는 탑이 있고 생 담쟁이\
덩굴로 만든 가느다란 턱수염과 대리석 수영장이 새것으로 덮여 있었다. 40에이커 이\
상의 잔디밭과 정원. 개츠비의 저택이었다. 아니, 차라리 내가 미스터 개츠비를 몰랐기에\
그 이름의 신사가 사는 저택이었다. 내 집은 눈에 거슬렸지만 작은 눈에 거슬렸으며 간\
과되었으므로 물이 보이고 이웃의 잔디가 부분적으로 보이고 위로가 되는 백만장자들의 근접성\
을 한 달에 80달러로 볼 수 있었습니다. \
예의 베이 건너편에는 패셔너블한 이스트 에그의 하얀 궁전이 물가를 따라 반짝이고 있었\
고, 여름의 역사는 내가 Tom Buchanans와 저녁 식사를 하기 위해 차를 몰고 그곳으로\
간 저녁부터 시작되었습니다. Daisy는 한 번 제거된 내 두 번째 사촌이었고 Tom\
은 대학에서 알고 있었습니다. 그리고 전쟁 직후 나는 시카고에서 그들과 함께 이틀을 보냈습니다.\
그녀의 남편은 다양한 신체적 업적 중에서 뉴 헤이븐에서 축구를 한 가장 강력한 목표 중 하나였습니\
다. 어떤 의미에서 국가적 인물, 21세에 극도로 제한적인 우수성에 도달한 남자 중 한 명입\
니다. 안티 클라이맥스. 그의 가족은 엄청나게 부유했지만(대학 시절에도 돈이 있는 그의 자\
유는 비난받아 마땅한 문제였습니다) 이제 그는 시카고를 떠나 숨이 멎을 듯한 방식으로 동부\
로 왔습니다. Lake Forest의 폴로 조랑말. 우리 세대의 남자가 그렇게 할 만큼 부유하다는\
것을 깨닫는 것은 어려웠습니다.', min_length=min_length, max_length=max_length)

[{'summary_text': '김계우는 3대 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중 중'}]

In [184]:
from transformers import pipeline

#hub_model_id = 'KETI-AIR/ke-t5-small-ko'
hub_model_id = 'data/test_2'
summarizer = pipeline("summarization", model=hub_model_id)

In [189]:
min_length=30
max_length=512

summarizer('그는 더 이상 말을 하지 않았지만 우리는 항상 유난히 절제된 방식으로 의사소통을 해왔고 나는 그가\
그 이상을 의미한다는 것을 이해했습니다. 결과적으로 나는 모든 판단을 유보하는 경향이 있으\
며, 그 습관은 나에게 많은 호기심을 불러일으켰고 또한 나를 적지 않은 노련한 지루한 사람의\
희생자로 만들었습니다. 비정상적인 마음은 정상적인 사람에게 나타날 때 이 특성을 빠르게 감지하고\
부착합니다. 그래서 대학에서 저는 부당하게 정치인이라는 비난을 받았습니다. 남자들. 대부분\
의 비밀은 구하지 않은 것입니다. 은밀한 계시가 수평선에서 떨고 있다는 확실한 신호를 깨달\
았을 때 나는 자주 잠, 몰두 또는 적대적인 경솔함을 가장했습니다. 젊은 남성의 내밀한 폭로, 또는\
적어도 그들이 표현하는 용어는 대개 표절적이며 명백한 억압으로 손상되었습니다. 판단을\
유보하는 것은 무한한 희망의 문제입니다. 아버지가 속물스럽게 말했고 내가 속물스럽게 되풀이해서 말\
하듯이 기본적인 품위 감각은 태어날 때부터 불평등하게 나누어진다는 사실을 잊어버리면 여\
전히 무언가를 놓치는 것이 조금 두렵습니다.', min_length=min_length, max_length=max_length)

Your max_length is set to 512, but you input_length is only 186. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=93)


[{'summary_text': '나는 모든 판단을 유보하는 경향이 있으며, 그 습관은 흥미가 커짐에 따라 오랜 동안 비정상적인 마음이 발생하여 오랜 동안 비정상적인 기억을 기억하여 부당한 정치인이라는 비난을 받았다.'}]

### validataion data로 테스트

In [207]:
def print_summary(idx):
    max_length = 512
    review = dataset["validation"][idx]["summary"]
    summary = summarizer(dataset["validation"][idx]["passage"], max_length=max_length)[0]["summary_text"]
    print(f"'>>> Review: {review}'")
    print(f"\n'>>> Summary: {summary}'")

In [208]:
max_length = 512
summarizer(dataset["validation"][1]["passage"],max_length=max_length)

Your max_length is set to 512, but you input_length is only 380. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=190)


[{'summary_text': '침해자의 사용의 정도 열한째 요소는 침해자의 침해된 특허발명의 실시가 아닌 침해된 특허의 사용가치를 증명할 수 있는 증거이다. 그 입증 증거의 폭은 넓어, 판매, 이익, 판매, 주가상승, 다른 여러 가지 침해된 특허에 귀속될 수'}]

In [209]:
print_summary(0)

Your max_length is set to 512, but you input_length is only 313. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=156)


'>>> Review: 특허제품의 유용성과 유리함은 유사한 결과를 실현하기 위해 사용되는 구 방식이나 기기에 대한 특허제품의 실용성 및 이점으로 합리적 실시료는 특허발명이 실시될 때 얻어지는 이득에 기초하여야 한다는 관념을 반영하고 만일 협상이 깨진다면 실시료 상당액은 실시료 약정에 응할 이유를 가져야 하는 경제적 가치의 할당된 비율을 나타내야만 한다는 것이다. '

'>>> Summary: 특허제품의 수익성과 인기는 특허발명의 실시로 제조된 제품의 발전된 수익성 및 그 사업상의 성공도, 현재 인기도이다. 특허발명의 성질, 특허발명의 성질, 특허발명의 성질과 특징, 특허발명의 성질과 특징 열째 요소는 특허발'


In [210]:
print_summary(10)

Your max_length is set to 512, but you input_length is only 307. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=153)


'>>> Review: 실시료 상당액은 특허발명의 실시에 대하여 통상 받을 수 있는 금액에 상당하는 액을 특허권자 또는 전용실시권자가 받은 손해의 액으로 하여 손해배상을 청구하는 것으로 특허발명을 실시하지 않고 있는 특허권자라고 하더라도 침해자를 발견한 경우에는 실시료 상당액을 손해액으로 삼아 배상을 청구할 수 있다는 것이다. '

'>>> Summary: 침해자의 이익반환 청구는 권리자 실시필요설에서는 견해 일치를 통해 권리자의 실시가 없는 경우, 권리자의 특허실시는 그 요건이 아니다. 특허발명을 실시하지 않고 있는 특허권자나 침해자를 발견한 경우, 실시료 상당액을 손해액으로 삼아 배상을'


In [211]:
print_summary(100)

Your max_length is set to 512, but you input_length is only 275. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=137)


'>>> Review: 불법행위가 있어도 피해자에게 손해가 전혀 없다면 손해배상의 청구는 불가능한 것이 우리나라 대원칙인데, 판례는 부동산의 불법점거라는 사실이 없었다고 하여도 부동산소유자에게 임료 상당의 이익이나 특별한 사정이 있는 때에는 손해배상을 청구할 수 없다고 판시하고 있고, 특허발명의 경우도 사실상의 이용가능성이 없었다면 특허침해가 있었다고 하더라도 손해배상이 불가능한 경우가 있다. '

'>>> Summary: 타인의 불법행위에 대한 손해배상 청구는 불가능하며, 특허발명의 경우에도 사실상의 이용가능성이 없었다면 손해배상이 불가능하다. 따라서 부동산의 불법점거에 관한 판례는 이를 잘 설명하고 있다.'
